# Reddit NBA scrape
This notebook has code to scrape comments for one month of reddit. Original version used a Reddit API, but now I use the `pushshift` API. Pushshift typically yields ~300k comments / month for a month during the season.

In [1]:
#import praw # for direct reddit pull
import time
import requests
import importlib
from datetime import datetime
from ast import literal_eval
import json
import pandas as pd

In [2]:
import scrape_reddit_comments as src
import scrape_player_data as spd

nba_coach_df.to_csv('d:/data/sentiment_sports/')

In [3]:
%%time
league = 'nba'
year = 2015
month = 12
month_df, submissions, comments = src.get_month_pushshift(year,month,30, league)

Downloaded 11015 submissions
Made dataframe of shape (11015, 9)


SSLError: HTTPSConnectionPool(host='api.pushshift.io', port=443): Max retries exceeded with url: /reddit/search/comment/?subreddit=nba&size=500&before=25791h&after=25792h (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'SSL23_GET_SERVER_HELLO', 'sslv3 alert handshake failure')],)",),))

In [4]:
month_df.sample(10)

NameError: name 'month_df' is not defined

### Save / load
Something weird is going on where is throws a `UnicodeDecodeError` when writing to a gzipped file

In [ ]:
month_df.to_csv(f'd:/data/sentiment_sports/{league}_reddit_comments/{year}{month:02}-comments_submissions.tsv',
                sep = '\t', encoding = 'utf-8',index = False)

## Scrape and combine covariate data
### Coach data
#### NBA

In [ ]:
%%time
importlib.reload(spd)
nba_coach_df = spd.scrape_nba_coaches()
nba_coach_df.to_csv('d:/data/sentiment_sports/covariates/nba_coach_performance.tsv', sep='\t', index=False)

#### NFL

In [4]:
importlib.reload(spd)
nfl_coach_df = spd.scrape_nfl_coaches()
nfl_coach_df.to_csv('d:/data/sentiment_sports/covariates/nfl_coach_performance.tsv', sep='\t', index=False)

Could not load URL https://www.pro-football-reference.com/coaches/ParkBu0.htm
Could not load URL https://www.pro-football-reference.com/coaches/ShawBu0.htm
Could not load URL https://www.pro-football-reference.com/coaches/LeeHJi0.htm
Could not load URL https://www.pro-football-reference.com/coaches/HickRe0.htm
Could not load URL https://www.pro-football-reference.com/coaches/StraRe0.htm
Could not load URL https://www.pro-football-reference.com/coaches/McMiBo0.htm
Could not load URL https://www.pro-football-reference.com/coaches/StarLo0.htm
Could not load URL https://www.pro-football-reference.com/coaches/RamsBu0.htm
Could not load URL https://www.pro-football-reference.com/coaches/GansFr0.htm
Could not load URL https://www.pro-football-reference.com/coaches/MacPDi0.htm
Could not load URL https://www.pro-football-reference.com/coaches/RobeJD0.htm
Could not load URL https://www.pro-football-reference.com/coaches/TurnBu0.htm
Could not load URL https://www.pro-football-reference.com/coache

In [6]:
# then hand edit it
nfl_coach_df.query('year >= 2013')[['Coach']].drop_duplicates().to_csv('d:/data/sentiment_sports/covariates/nfl_coach_race.tsv', sep='\t', index=False)

#### Create separate .tsv for coach race

### Use PRAW and reddit API to get lots of comments
This no longer works due to update of Reddit API

In [2]:
client_id = 'wgPHyF8Ogio_dA'
secret = '3BkmoB6wVUWzF0wsOSkIxnI-guo'
user_agent = 'r/nba race sentiment 0.1 by /u/Umiy'

In [3]:
r = praw.Reddit(user_agent=user_agent, client_id=client_id, client_secret=secret)
r_nba = r.subreddit('nba')

In [138]:
def parse_submission( submission):
    text = submission.title + '. ' + submission.selftext
    creation_date = submission.created
    author = submission.author #.name for PRAW
    flair = submission.author_flair_text
    score = submission.score
    return (text, creation_date, author, flair, score)

In [140]:
def parse_comment( comment):
    if hasattr(comment, 'body') and comment.author != None:
        text = comment.body
        creation_date = comment.created
        author = comment.author
        flair = comment.author_flair_text
        score = comment.score
        return (text, creation_date, author, flair, score)
    return ('', 1, '', '', 0)

In [6]:
def get_month(year, month, month_length):
    ''' Get comments and submissions for one month. takes ~ 1h 15 minutes.
    '''
    data_col = ['text', 'timestamp', 'user', 'flair']
    print('Downloading submssions for {}-{}'.format(year, month))
    month_submissions = [list(r_nba.submissions(datetime(year,month,day).timestamp(), datetime(year,month,day+1).timestamp() )) for day in range(1,month_length)]
    month_submissions = [x for day_submissions in month_submissions for x in day_submissions]
    
    print('Downloaded {} submissions'.format(len(month_submissions)))
    ops = [ parse_submission(submission) for submission in month_submissions]
    submission_df =(pd.DataFrame(ops, columns=data_col)
                      .assign(source = lambda x: 'submission') )
    print('Made dataframe of shape {}'.format(submission_df.shape) )
    
    print('Downloading comments (this could take an hour)')
    comments_list = [submission.comments.list() for submission in month_submissions if hasattr(submission, 'comments')]
    comments = [ parse_comment(comment) for comments in  comments_list for comment in comments]
    print('Downloaded {} comments'.format(len(comments) ) )
    comment_df = (pd.DataFrame(comments, columns=data_col)
                    .assign(source = lambda x: 'comment') )
    return pd.concat([submission_df, comment_df]), ops, comments

In [ ]:
%%time
# this no longe works 
month_df, submissions, comments = get_month(2017, 11, 30)